In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression

import torch
import torch.optim as optim
import torch.nn as nn

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
true_b = 1 # parameter setting
true_w = 2 #true parameter
N = 100 #n_size

# Data Generation
np.random.seed(42)
x = np.random.rand(N, 1) #random x number n size
epsilon = (.1 * np.random.randn(N, 1)) # error term 1/10 of the true number..?
y = true_b + true_w * x + epsilon 


# Shuffles the indices
idx = np.arange(N) 
np.random.shuffle(idx) #shuffled indices

# Uses first 80 random indices for train
train_idx = idx[:int(N*.8)] #select first 80% as training
# Uses the remaining indices for validation
val_idx = idx[int(N*.8):] #select the last 20% as validation

#print(train_idx, val_idx)

# Generates train and validation sets
x_train, y_train = x[train_idx], y[train_idx] # training index are used to retrieve datapoint
x_val, y_val = x[val_idx], y[val_idx] #training index are used to retrieve datapoints.


x_train_tensor = torch.as_tensor(x_train).float().to(device)
y_train_tensor = torch.as_tensor(y_train).float().to(device)

y_train_tensor.type()

'torch.FloatTensor'

Use Parameter class to initialize the model parameter

https://pytorch.org/docs/stable/generated/torch.nn.parameter.Parameter.html#parameter


In [4]:

class ManualLinearRegression(nn.Module):
  def __init__(self):
    super().__init__()
    self.b = nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float, device = device))
    self.w = nn.Parameter(torch.randn(1, requires_grad=True, dtype = torch.float, device= device))

  def forward(self, x):
    return self.b + self.w * x



Then this constructed parameter can be accessed using [.parameters()](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.parameters)


In [5]:
torch.manual_seed(42) # Creates a "dummy" instance of our ManualLinearRegression model 
dummy = ManualLinearRegression() 
list(dummy.parameters())

[Parameter containing:
 tensor([0.3367], requires_grad=True), Parameter containing:
 tensor([0.1288], requires_grad=True)]

We can get the current state of the parameter;
https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.state_dict


In [6]:
dummy.state_dict()

OrderedDict([('b', tensor([0.3367])), ('w', tensor([0.1288]))])

In [7]:
torch.manual_seed(42)
dummy = ManualLinearRegression().to(device)

now we use the model.

In [8]:
lr = 0.1

torch.manual_seed(42)
model = ManualLinearRegression().to(device)

optimizer = optim.SGD(model.parameters(), lr = lr) #here we pass on model.parameters.
loss_fn = nn.MSELoss(reduction = 'mean')

n_epochs = 1000

for epoch in range(n_epochs):
  model.train()

  yhat = model(x_train_tensor)
  loss = loss_fn(yhat, y_train_tensor)

  loss.backward()

  optimizer.step()
  optimizer.zero_grad()

print(model.state_dict())

OrderedDict([('b', tensor([1.0235])), ('w', tensor([1.9690]))])


## Nested models

In [9]:
linear = nn.Linear(1,1)
linear

Linear(in_features=1, out_features=1, bias=True)

In [10]:
linear.state_dict()

OrderedDict([('weight', tensor([[-0.2191]])), ('bias', tensor([0.2018]))])

In [11]:
class MyLinearRegression(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(1, 1)

  def forward(self, x):
    self.linear(x)
    

In [13]:
torch.manual_seed(42)
dummy = MyLinearRegression().to(device)
list(dummy.parameters())



[Parameter containing:
 tensor([[0.7645]], requires_grad=True), Parameter containing:
 tensor([0.8300], requires_grad=True)]

model.state_dict() will return parameter nme and values.

In [14]:
dummy.state_dict()

OrderedDict([('linear.weight', tensor([[0.7645]])),
             ('linear.bias', tensor([0.8300]))])

## Sequential models
The reason we use class is to expand this to larger models.

In [15]:
torch.manual_seed(42)

model = nn.Sequential(nn.Linear(1, 1)).to(device)

model.state_dict()

OrderedDict([('0.weight', tensor([[0.7645]])), ('0.bias', tensor([0.8300]))])

In [16]:
model2 = nn.Sequential(nn.Linear(1,1),
                       nn.Linear(1,2)).to(device)

model2.state_dict

<bound method Module.state_dict of Sequential(
  (0): Linear(in_features=1, out_features=1, bias=True)
  (1): Linear(in_features=1, out_features=2, bias=True)
)>

## Adding more layers.
- Use Sequential()
- Use add_module()

In [17]:
torch.manual_seed(42)
model = nn.Sequential(nn.Linear(3,5),
                      nn.Linear(5,1)).to(device)
model.state_dict()

OrderedDict([('0.weight', tensor([[ 0.4414,  0.4792, -0.1353],
                      [ 0.5304, -0.1265,  0.1165],
                      [-0.2811,  0.3391,  0.5090],
                      [-0.4236,  0.5018,  0.1081],
                      [ 0.4266,  0.0782,  0.2784]])),
             ('0.bias', tensor([-0.0815,  0.4451,  0.0853, -0.2695,  0.1472])),
             ('1.weight',
              tensor([[-0.2060, -0.0524, -0.1816,  0.2967, -0.3530]])),
             ('1.bias', tensor([-0.2062]))])

In [18]:
torch.manual_seed(42)



model3 = nn.Sequential()
model3.add_module('layer1', nn.Linear(3,5))
model3.add_module('layer2', nn.Linear(5,1))
model3.to(device)



Sequential(
  (layer1): Linear(in_features=3, out_features=5, bias=True)
  (layer2): Linear(in_features=5, out_features=1, bias=True)
)